In [0]:
print("Hello")

Hello

In [0]:
# driver program
data = [10,20,30,40] # data, list
# sc.<<TAB>>
rdd = sc.parallelize(data) # distribute the data across cluster
# max - action function, it get results, create task, execute task on workers, get the result
print(rdd.max())

40

In [0]:
data = [1,2,3,4,5,6,7,8,9,10]
# RDDs are immutable
rdd = sc.parallelize(data) # data input, lazy won't run until action called

rdd2 = rdd.map (lambda n: n * 3) # tranformation, lazy, , lazy won't run until action called

rdd3 = rdd2.filter (lambda n: n % 2 == 0) # transform, take only even numbers, ignore odd number

# collect data to driver.. NOT LAZY method like transform.
# action 1, job 1
result = rdd.collect() # action method, create job, task, stages, DAG, run the tasks on worker, finally get result

# collect result of rdd2 which is multipling each number by 3
print(result)

# action 2, job 2
result2 = rdd2.collect()  # action, perform and run transformation, task....

print(result2)

print(rdd2.toDebugString())

print (rdd3.collect()) # action, even numbers

print(rdd3.toDebugString())

# string with """ means here you can write multiline text, also comments """
"""
Top to Bottom
  rdd 
    - rdd2
        - rdd3
 
"""


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[3, 6, 9, 12, 15, 18, 21, 24, 27, 30]
b'(8) PythonRDD[15] at collect at <command-4158140795229798>:17 []\n | ParallelCollectionRDD[14] at readRDDFromFile at PythonRDD.scala:332 []'
[6, 12, 18, 24, 30]
b'(8) PythonRDD[16] at collect at <command-4158140795229798>:23 []\n | ParallelCollectionRDD[14] at readRDDFromFile at PythonRDD.scala:332 []'
Out[10]: '\nTop to Bottom\n rdd \n - rdd2\n - rdd3\n \n'

In [0]:
# is rdd processing data from begining or not... - Yes
data = [1,2,3]

rdd = sc.parallelize(data)

# run on worker, lmabda with map (lambda n : n * 3) run on cluster/worker
def multiply(n):
  print (n, "*", 3, "called")
  return n * 3

def filterEven(n):
  print (n, "%", 2, "fillter called")
  return n  % 2 == 0

# multiply, lambda, are run inside spark cluster, not on the spark driver
# print output goes background
rdd2 = rdd.map (multiply) #  NO TASK, No Partitions, no memory created
# run on driver
rdd3 = rdd2.filter (filterEven)   #  NO TASK, No Partitions, no memory created

rdd4 = rdd.map (lambda n : n * 4)


# rdd3.collect()
# run on driver
print(rdd3.collect()) # calling action on rdd first time, sync call, creation memory, tasks
#print(rdd3.collect()) # calling action on rdd second time
 
print(rdd3.min()) # action
print(rdd3.max()) # action

[6]

In [0]:
# Default partitions
data = [1,2,3,4,5,6]

rdd = sc.parallelize(data)

print(sc.defaultMinPartitions) # default partition given as 2
print(sc.defaultParallelism) # 8 parallel tasks can be performed

print("rdd parts ", rdd.getNumPartitions())

print ("partitiion data")

# colelct() collect data from all paritions and club them together

# glom() this collect data from individual paritions, bring them as is
# glom() returning an rdd which is partition specific? method
partData = rdd.glom().collect()
for d in partData:
  print (d)
  

2
8
rdd parts 8
partitiion data
[]
[1]
[2]
[3]
[]
[4]
[5]
[6]

In [0]:
# limit paritions during input stage

data = [1,2,3,4,5,6]

rdd = sc.parallelize(data, 4)

print("rdd parts ", rdd.getNumPartitions())

print ("partitiion data")

# colelct() collect data from all paritions and club them together

# glom() this collect data from individual paritions, bring them as is
# glom() returning an rdd which is partition specific? method
partData = rdd.glom().collect()
for d in partData:
  print (d)

rdd parts 4
partitiion data
[1]
[2, 3]
[4]
[5, 6]

In [0]:
# limit paritions using reparition

data = [1,2,3,4,5,6]

rdd = sc.parallelize(data)

print("rdd parts ", rdd.getNumPartitions()) # 8

print ("rdd partitiion data")

# colelct() collect data from all paritions and club them together

# glom() this collect data from individual paritions, bring them as is
# glom() returning an rdd which is partition specific? method
partData = rdd.glom().collect()
for d in partData:
  print (d)
  

# reduce the paritions into 2, all data from rdd's 8 paritions, here filled in two paritions
rdd2 = rdd.repartition(2)
print("rdd2 parts ", rdd2.getNumPartitions()) # 2

print ("rdd2 partitiion data")
partData = rdd2.glom().collect()
for d in partData:
  print (d)


rdd parts 8
rdd partitiion data
[]
[1]
[2]
[3]
[]
[4]
[5]
[6]
rdd2 parts 2
rdd2 partitiion data
[2, 3, 5]
[1, 4, 6]

In [0]:
# 4 core cpus, If HT enabled, split into 8 logical cores, v.cores
# number of v.cores = 8

# IO - read csv - Disk IO, Network IO, Hadoop, S3, Latency
# RAM speed slower than CPU - IO

# Parallel Tasks = v.cores * 2 or 3 times 

# community edition 2 v.cores * 4 = 8 parallel tasks

In [0]:
# RDD Cache
# is rdd processing data from begining or not... - Yes
data = [1,2,3]

rdd = sc.parallelize(data)

# run on worker, lmabda with map (lambda n : n * 3) run on cluster/worker
def multiply(n):
  print (n, "*", 3, "called")
  return n * 3

def filterEven(n):
  print (n, "%", 2, "fillter called")
  return n  % 2 == 0

rdd2 = rdd.map (multiply) 

# cache internally calls persist function with MEMORY_AND_DISK
rdd2.cache() # Lazy, when called with action, the partition data is cached in worker
print("is_cached", rdd2.is_cached)

rdd3 = rdd2.filter (filterEven)   
rdd4 = rdd3.map (lambda n : n * 4)

# run on driver
print(rdd4.collect()) # calling action on rdd first time, sync call, 
print(rdd3.min()) # action
print(rdd3.max()) # action

rdd2.unpersist() # clear the cache
# sc.stop()

is_cached True
[24]
6
6